In [1]:
import os
from utils import helpers

os.chdir('..')

In [2]:
import re
from heapq import nlargest

In [3]:
input = helpers.get_input(2016, 4)

Found a datafile!


In [22]:
for i in range(123-97):
    print(chr(i+97), i)

print(ord('z') + 343 - 97)

print(368%26)

a 0
b 1
c 2
d 3
e 4
f 5
g 6
h 7
i 8
j 9
k 10
l 11
m 12
n 13
o 14
p 15
q 16
r 17
s 18
t 19
u 20
v 21
w 22
x 23
y 24
z 25
368
4


In [23]:
def decipher(string:str, num:int) -> str:
    decoded = []
    for char in string:
        shifted = ord(char) - 97 + num
        decoded.append(chr(shifted%26))
    return "".join(decoded) 

In [24]:
print(decipher("qzmtzixmtkozyivhz", 343))

TypeError: sequence item 0: expected str instance, int found